In [ ]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{B}i$ for $i\in\{0,\cdots,n\}$ is the number of boats on the western shore after the 
  $i^{\textrm{th}}$ crossing.

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [ ]:
def flatten(LoL):
    return [x for L in LoL for x in L]

In [ ]:
flatten([[1,2], [3,4]])

The function `start` takes three integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `True` in the initial state where everybody is on the western shore.

In [ ]:
def start(M, C, B):
    return M == 3 and C == 3 and B == 1

The function `goal` takes three integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `True` in the state where everybody is on the eastern shore
and hence nobody is left on the western shore.

In [ ]:
def goal(M, C, B):
    return M == 0 and C == 0 and B == 0

The function `invariant` takes three integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `True` if there is no problem on either shore of the river.  
There is no problem if any one of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. 
  $\texttt{M} = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M} = 3$.
  Then there are no missionaries on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of 
  infidels on that side, i.e. $\texttt{M} = \texttt{C}$.  Then the numbers of 
  missionaries and infidels have to match on the eastern shore as well.
  
Furthermore, as there is just one boat, the number $\texttt{B}$ of boats is at most one.

In [ ]:
def invariant(M, C, B):
    return (M == 0 or M == 3 or M == C) and B <= 1

The function `transition` takes 6 arguments:
* `M𝛼` is the number of missionaries on the western shore before the crossing.
* `C𝛼` is the number of infidels on the western shore before the crossing.
* `B𝛼` is the number of boats on the western shore before the crossing. 
* `M𝛽` is the number of missionaries on the western shore after the crossing.
* `C𝛽` is the number of infidels on the western shore after the crossing.
* `B𝛽` is the number of infidels on the western shore after the crossing.
  

The function call `transition(M𝛼, C𝛼, B𝛼, M𝛽, C𝛽, B𝛽)` returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the crossing.  Note that if `B𝛼 == 1`, then the boat travels from the western shore to the eastern shore.  If `B𝛼 == 0`, the boat travels from the eastern shore to the western shore.

The condition 
```
   1 <= M𝛼 - M𝛽 + C𝛼 - C𝛽 <= 2
```
ensures that the boat has at least one and at most two passengers when travelling from west to east.  The conditions
```
M𝛽 <= M𝛼 and C𝛽 <= C𝛼
```
ensures that the number of missionaries and infidels on the western shore does not increase when the boat travels from the western shore to the eastern shore.

In [ ]:
def transition(M𝛼, C𝛼, B𝛼, M𝛽, C𝛽, B𝛽):
    if B𝛽 != 1 - B𝛼:  # boat has to change sides during a crossing
        return False
    if B𝛼 == 1:       # boat travels from west to east
        return (1 <= M𝛼 - M𝛽 + C𝛼 - C𝛽 <= 2 and M𝛽 <= M𝛼 and C𝛽 <= C𝛼)
    else:
        return (1 <= M𝛽 - M𝛼 + C𝛽 - C𝛼 <= 2 and M𝛽 >= M𝛼 and C𝛽 >= C𝛼)

The function `missionaries_CSP` creates a CSP that tries to solve the problem with `n` crossings.

In [ ]:
def missionaries_CSP(n):
    "Returns a CSP encoding the problem."
    Lists        = [[f'M{i}', f'C{i}', f'B{i}'] for i in range(n+1)]
    Variables    = flatten(Lists)
    Values       = { 0, 1, 2, 3 }
    Constraints  = {  'start(M0, C0, B0)'      }  # start state
    Constraints |= { f'goal(M{n}, C{n}, B{n})' }  # goal state
    for i in range(n):
        Constraints |= { f'invariant(M{i}, C{i}, B{i})' }
        Constraints |= { f'transition(M{i}, C{i}, B{i}, M{i+1}, C{i+1}, B{i+1})' }
    return Variables, Values, Constraints

In [ ]:
missionaries_CSP(3)

In [ ]:
%run 02-Backtracking-Constraint-Solver.ipynb

The function `find_solution` computes a solution to the problem of saving the infidels.

In [ ]:
def find_solution():
    n = 1
    while True:
        print(n)
        CSP = missionaries_CSP(n)
        Solution = solve(CSP)
        if Solution != None:
            return n, Solution
        n += 2

On my desktop computer (2023 MacStudio with M2 Max processor) it takes about 1 second to solve the problem. 

In [ ]:
%%time
n, Solution = find_solution()
n, Solution

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        M = Solution[f'M{i}']
        C = Solution[f'C{i}']
        B = Solution[f'B{i}']
        print('😇' * M + '🥷' * C + ' ' * 28 + '😇' * (3 - M) + '🥷' * (3 - C))
        if B == 1:
            MB = Solution[f'M{i}'] - Solution[f'M{i+1}']
            CB = Solution[f'C{i}'] - Solution[f'C{i+1}']
            print(' ' * 12 + '>>> ' + '😇'*MB + '🥷'*CB + ' >>>')
        elif i + 1 < n:
            MB = Solution[f'M{i+1}'] - Solution[f'M{i}']
            CB = Solution[f'C{i+1}'] - Solution[f'C{i}']
            print(' ' * 12 + '<<< ' + '😇'*MB + '🥷'*CB + ' <<<')

In [ ]:
show_solution(Solution, n)